# Download and Save the Model

To save this model so that you can use it from various locations, including other notebooks or the model server, upload it to s3-compatible storage.

In [ ]:
!pip install boto3 botocore

## Install the required packages and define a function for the upload

## Download from Huggingface

In [ ]:
# !git clone https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2
!hfd mistralai/Mistral-7B-Instruct-v0.2 --exclude .bin --hf_username $HF_USERNAME --hf_token $HF_TOKEN --tool aria2c -x 8

## Helper functions for upload

In [3]:
import os
import boto3
import botocore

aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
endpoint_url = os.environ.get('AWS_S3_ENDPOINT')
region_name = os.environ.get('AWS_DEFAULT_REGION')
bucket_name = os.environ.get('AWS_S3_BUCKET')

session = boto3.session.Session(aws_access_key_id=aws_access_key_id,
                                aws_secret_access_key=aws_secret_access_key)

s3_resource = session.resource(
    's3',
    config=botocore.client.Config(signature_version='s3v4'),
    endpoint_url=endpoint_url,
    region_name=region_name)

bucket = s3_resource.Bucket(bucket_name)

#upload the model directory without git
def upload_directory_to_s3(local_directory, s3_prefix):
    for root, dirs, files in os.walk(local_directory):
        for filename in files:
            file_path = os.path.join(root, filename)
            relative_path = os.path.relpath(file_path, local_directory)
            if ".git" in relative_path:
                continue
            s3_key = os.path.join(s3_prefix, relative_path)
            print(f"{file_path} -> {s3_key}")
            bucket.upload_file(file_path, s3_key)


def list_objects(prefix):
    filter = bucket.objects.filter(Prefix=prefix)
    for obj in filter.all():
        print(obj.key)

## Check the Storage Bucket

In your S3 bucket, under the `models` upload prefix, run the `list_object` command. As best practice, to avoid mixing up model files, keep only one model and its required files in a given prefix or directory. This practice allows you to download and serve a directory with all the files that a model requires. 

If this is the first time running the code, this cell will have no output or the fraud model from the predictive AI/ML exercise.


In [ ]:
list_objects("models")

## Upload and check again

In [ ]:
upload_directory_to_s3("Mistral-7B-Instruct-v0.2", "models/Mistral-7B-Instruct-v0.2")

To confirm this worked, run the `list_objects` function again:

Use the function to upload the `models` folder in a rescursive fashion:

In [ ]:
list_objects("models")

Push to Model Registry

In [ ]:
!pip install model-registry=="0.2.14"
#!pip install kserve=="0.13"

from model_registry import ModelRegistry

In [ ]:
# grab the url from the annotations in service

registry = ModelRegistry(
    server_address="https://test-rest.apps.cluster-rr2bp.rr2bp.sandbox199.opentlc.com",
    author="test"
)

In [ ]:
rm = registry.register_model(
    "Mistral",
    "minio-api-minio.apps.cluster-rr2bp.rr2bp.sandbox199.opentlc.com/models/models/Mistral-7B-Instruct-v0.2",
    model_format_name="sklearn",
    model_format_version="1",
    version="v1",
    description="Model for code Assistance",
    metadata={
        "accuracy": 3.14,
        "license": "BSD 3-Clause License",
    }
)